In [1]:
import os
##指向第四张卡，前三张在做训练
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import paddle
paddle.utils.run_check()


Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle works well on 1 GPUs.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [2]:
from paddlenlp.embeddings import TokenEmbedding
token_embedding = TokenEmbedding(embedding_name = "w2v.wiki.target.word-word.dim300")

[2021-06-09 16:38:18,752] [    INFO] - Loading token embedding...
[2021-06-09 16:38:22,118] [    INFO] - Finish loading embedding vector.
[2021-06-09 16:38:22,122] [    INFO] - Token Embedding info:             
Unknown index: 352217             
Unknown token: [UNK]             
Padding index: 352218             
Padding token: [PAD]             
Shape :[352219, 300]


In [3]:
print(token_embedding)

Object   type: TokenEmbedding(352219, 300, padding_idx=352218, sparse=False)             
Unknown index: 352217             
Unknown token: [UNK]             
Padding index: 352218             
Padding token: [PAD]             
Parameter containing:
Tensor(shape=[352219, 300], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[ 0.15078500,  0.12059100, -0.22020400, ..., -0.53268701, -0.06691400, -0.10072800],
        [-0.25135499,  0.23474200, -0.16972800, ..., -0.52722102, -0.22953101,  0.18078101],
        [ 0.20782000,  0.14142500, -0.33728099, ..., -0.55558801,  0.04756100, -0.07089700],
        ...,
        [ 0.00247700, -0.00236600, -0.02842000, ..., -0.02777100, -0.01926100,  0.00767400],
        [-0.00824524, -0.00958661,  0.01343321, ...,  0.01424389, -0.03027157, -0.00872602],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,  0.        ,  0.        ]])


In [7]:
##查询词向量
test_token_embedding = token_embedding.search("祖国")
print(test_token_embedding)

[[-0.067538  0.189535  0.049175 -0.109212 -0.242945 -0.161946 -0.35808
  -0.209951  0.05232  -0.104802  0.004568  0.040071  0.090112  0.121372
   0.245141  0.353702 -0.003359 -0.080915 -0.048213  0.252324 -0.022077
   0.197302  0.149266  0.39154   0.554964 -0.640223 -0.330378 -0.140783
  -0.294932 -0.0542    0.38516   0.009788  0.080107 -0.315227  0.384387
   0.257482  0.411435 -0.061848 -0.176585 -0.320802 -0.06256  -0.021686
   0.185725 -0.030455 -0.040159  0.320913  0.305943 -0.035203 -0.29851
   0.047758 -0.159635  0.043715 -0.1643    0.112271 -0.296435  0.132372
  -0.234751 -0.306376 -0.22427  -0.186935 -0.08255  -0.211002  0.198403
  -0.095023 -0.513217 -0.029123 -0.25614   0.205883  0.660782  0.12478
  -0.162462 -0.568987 -0.178057 -0.001162  0.201736  0.273989  0.209728
  -0.338654  0.088953  0.010972  0.59236  -0.478215 -0.365565 -0.127514
   0.039382  0.091705 -0.540819  0.344238  0.322735  0.175959 -0.049957
   0.032881  0.322046 -0.151226  0.168387  0.623019 -0.097401 -0.04

In [4]:
##计算两个词之间的相关性问题（余弦距离 欧式距离等）
score1 = token_embedding.cosine_sim("祖国","爱国")
score2 = token_embedding.cosine_sim("男孩", "女孩")
print('score1:', score1)
print('score2:', score2)

score1: 0.4686648
score2: 0.597433


In [5]:
##采用百度自家工具visual显示词向量分布,当然还可以采用sklean高维到低维显示 Sklearn中TSNE tensorboard也是可以可视化
labels = token_embedding.vocab.to_tokens(list(range(0, 10000)))
test_token_embedding = token_embedding.search(labels)
from visualdl import LogWriter

with LogWriter(logdir='./token_hidi') as writer:
    writer.add_embeddings(tag='test', mat=[i for i in test_token_embedding], metadata=labels)

In [ ]:
![test_visualdl](images/test_visualdl.png)

In [6]:
import paddle
import paddle.nn as nn
import paddlenlp


class BoWModel(nn.Layer):
    def __init__(self, embedder):
        super().__init__()
        self.embedder = embedder
        emb_dim = self.embedder.embedding_dim
        self.encoder = paddlenlp.seq2vec.BoWEncoder(emb_dim)
        self.cos_sim_func = nn.CosineSimilarity(axis=-1)

    def get_cos_sim(self, text_a, text_b):
        text_a_embedding = self.forward(text_a)
        text_b_embedding = self.forward(text_b)
        cos_sim = self.cos_sim_func(text_a_embedding, text_b_embedding)
        return cos_sim

    def forward(self, text):
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)

        # Shape: (batch_size, embedding_dim)
        summed = self.encoder(embedded_text)

        return summed

model = BoWModel(embedder=token_embedding)

In [7]:
from data import Tokenizer
tokenizer = Tokenizer()
tokenizer.set_vocab(vocab=token_embedding.vocab)

In [8]:
text_pairs = {}
with open("text_pair.txt", "r", encoding="utf8") as f:
    for line in f:
        text_a, text_b = line.strip().split("\t")
        if text_a not in text_pairs:
            text_pairs[text_a] = []
        text_pairs[text_a].append(text_b)

In [9]:
for text_a, text_b_list in text_pairs.items():
    text_a_ids = paddle.to_tensor([tokenizer.text_to_ids(text_a)])

    for text_b in text_b_list:
        text_b_ids = paddle.to_tensor([tokenizer.text_to_ids(text_b)])
        print("text_a: {}".format(text_a))
        print("text_b: {}".format(text_b))
        print("cosine_sim: {}".format(model.get_cos_sim(text_a_ids, text_b_ids).numpy()[0]))
        print()
        

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 多项式矩阵的左共轭积及其应用
cosine_sim: 0.8933111429214478

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 退化阻尼对高维可压缩欧拉方程组经典解的影响
cosine_sim: 0.8055297136306763

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: Burgers方程基于特征正交分解方法的数值解法研究
cosine_sim: 0.8413742780685425

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 有界对称域上解析函数空间的若干性质
cosine_sim: 0.8249683976173401

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 基于卷积神经网络的图像复杂度研究与应用
cosine_sim: 0.7288050651550293

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: Cartesian发射机中线性功率放大器的研究
cosine_sim: 0.729754626750946

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: CFRP加固WF型梁侧扭屈曲的几何非线性有限元分析
cosine_sim: 0.8004938960075378

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 基于线性CCD自适应成像的光刻机平台调平方法研究
cosine_sim: 0.7524572610855103

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 基于变分贝叶斯理论的图像复原方法研究
cosine_sim: 0.6843059062957764

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 网格资源分配中混合并行蚁群算法方式研究
cosine_

In [10]:
# 引入VisualDL的LogWriter记录日志
import numpy as np
from visualdl import LogWriter    
# 获取句子以及其对应的向量
label_list = []
embedding_list = []

for text_a, text_b_list in text_pairs.items():
    text_a_ids = paddle.to_tensor([tokenizer.text_to_ids(text_a)])
    embedding_list.append(model(text_a_ids).flatten().numpy())
    label_list.append(text_a)

    for text_b in text_b_list:
        text_b_ids = paddle.to_tensor([tokenizer.text_to_ids(text_b)])
        embedding_list.append(model(text_b_ids).flatten().numpy())
        label_list.append(text_b)


with LogWriter(logdir='./sentence_hidi') as writer:
    writer.add_embeddings(tag='test', mat=embedding_list, metadata=label_list)